In [2]:
import propka

In [ ]:
proka.run(
    pdb_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.pdb", 
    output_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.out",
    pH=7.4,
    forcefield="amber",
    ion_strength=0.1,
    temperature=298.15,
    n_threads=4,
    verbose=True)

NameError: name 'proka' is not defined

In [18]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [21]:
list(d.keys())

['CYS',
 'ASP',
 'SER',
 'GLN',
 'LYS',
 'ILE',
 'PRO',
 'THR',
 'PHE',
 'ASN',
 'GLY',
 'HIS',
 'LEU',
 'ARG',
 'TRP',
 'ALA',
 'VAL',
 'GLU',
 'TYR',
 'MET']

In [51]:
def auto_detect_types():
    #input4amber.pdb
    # source leaprc.protein.ff19SB
    # source leaprc.water.opc
    # source leaprc.lipid21
    #sed -i '1s/^/task goes here\n/' todo.txt

    forcefields = {'protein': 'protein.ff19SB', 'lipid': 'lipid21', 'dna': 'DNA.OL24', 'rna': 'RNA.OL3'}

    protein = ['CYS','ASP','SER','GLN','LYS','ILE','PRO','THR','PHE','ASN','GLY','HIS','LEU','ARG','TRP','ALA','VAL','GLU','TYR','MET']
    lipid = ['PC', 'PA', 'OL', 'CHL']
    nucleic = ['A','C','G']
    rna = ['U']
    dna = ['T']
    with open('../tests/input4amber.pdb', 'r') as f:
        lines = f.readlines()

    resnames = []
    
    ff_types = []

    for line in lines:

        l = line.split()

        if l[0] == 'END':
            break

        if l[0] in ['ATOM', 'HETATM']:
            resnames.append(l[3])

    resnames = list(set(resnames))



    for resname in resnames:
        if resname in protein:
            if 'protein' not in ff_types:
                ff_types.append('protein')
        elif resname in lipid:
            if 'lipid' not in ff_types:
                ff_types.append('lipid')
        # elif resname in nucleic:
        #     if 'nucleic' not in ff_types:
        #         ff_types.append('nucleic')
        elif resname in rna:
            if 'rna' not in ff_types:
                ff_types.append('rna')
        elif resname in dna:
            if 'dna' not in ff_types:
                ff_types.append('dna')
        else:
            print('type not found', resname, ff_types)

    leaprc = ''

    for ff_type in ff_types:
        leaprc += f'source leaprc.{forcefields[ff_type]}\\n'

            
    return leaprc

In [ ]:
leap = auto_detect_types()
run(f"sed -i '1s/^/{leap}/' tleap.in")

type not found G []
type not found LIG ['protein']
type not found A ['protein']
type not found C ['protein', 'rna']


In [43]:
import argparse
import subprocess
from pathlib import Path
import sys
import os
import shutil
import yaml

def run(cmd, shell=True, check=True):
    if isinstance(cmd, list):
        print(f"Running: {' '.join(str(x) for x in cmd)}")
    else:
        print(f"Running: {cmd}")
    subprocess.run(cmd, shell=shell, check=check)

In [55]:
run(f"sed -i '1s/^/{leap}/' tleap.in")

Running: sed -i '1s/^/source leaprc.protein.ff19SB\nsource leaprc.RNA.OL3\n/' tleap.in
